## Fill and save the template

In [ ]:
import sys
import os
import math
from pathlib import Path
import pandas as pd

from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
sys.path.append(str(PROJECT_ROOT.joinpath('src')))
print(f"Project root directory: {PROJECT_ROOT}")

from utils.neuron_bag import NeuronBag
from utils.gallery_filler import generate_gallery_json
from utils.rend_params import get_rend_params
from utils import olc_client
from utils.ol_color import OL_COLOR
from utils.ol_types import OLTypes

c = olc_client.connect(verbose=True)

In [ ]:
olt = OLTypes()
oli_list = olt.get_neuron_list()

In [ ]:
iter_counter = 0
for idx, row in oli_list.reset_index().sample(frac=1).iterrows():
    iter_counter += 1
    all_cell_dict = {}
    txt_pos = 0.92

    body_id_dict = {}


    a_bag = NeuronBag(cell_type=row['type'])
    body_id = a_bag.get_body_ids(1)[0]
    if isinstance(row['star_neuron'], int):
       body_id = row['star_neuron']

    camera_dict = get_rend_params('camera', 'whole_brain')
    scalebar_dict = get_rend_params('scalebar','whole_brain')

    if row['main_groups']=='OL_intrinsic':
        the_directory='flipbook_OL_intrinsic'
        the_color=OL_COLOR.OL_IN_SEQ.rgba[1]
        the_title='Optic Lobe Intrinsic Neurons (OLIN)'

    if row['main_groups']=='OL_connecting':
        the_directory='flipbook_OL_connecting'
        the_color=OL_COLOR.OL_CONN_SEQ.rgba[1]
        the_title='Optic Lobe Connecting Neurons (OLCN)'

    if row['main_groups']=='VPN':
        the_directory='flipbook_VPN'
        the_color=OL_COLOR.OL_VPN_SEQ.rgba[1]
        the_title='Visual Projection Neurons (VPN)'

    if row['main_groups']=='VCN':
        the_directory='flipbook_VCN'
        the_color=OL_COLOR.OL_VCN_SEQ.rgba[1]
        the_title='Visual Centrifugal Neurons (VCN)'
    
    if row['main_groups']=='other':
        the_directory='flipbook_other'
        the_color=OL_COLOR.OL_CB_OTHER_SEQ.rgba[1]
        the_title='Other Visual Neurons'

    group_dict = {}
    body_id_dict = {
        'type': row['type']
      , 'body_ids': [body_id]
      , 'body_color': the_color
      , 'text_position': [0.03, txt_pos]
      , 'text_align': 'l'
      , 'number_of_cells': a_bag.size
      , 'slice_width': 0 
    }
  
    group_dict[row['type']] = body_id_dict   

    generate_gallery_json(
        type_of_plot="Full-Brain"
      , description="flipbook"
      , type_or_group=row['type']
      , title=the_title
      , view='whole_brain'
      , list_of_ids=group_dict
      , neuropil_color=[]
      , camera=camera_dict
      , slicer={}
      , scalebar=scalebar_dict
      , n_vis={}
      , directory=the_directory
      , template="gallery-descriptions.json.jinja"
    )
    print(f"Json generation done for {row['type']}")
    
    # Stop if number of iterations exceed the environment variable `GALLERY_EXAMPLES`
    stop_after = os.environ.get('GALLERY_EXAMPLES')
    if stop_after:
        if stop_after := int(stop_after):
            if stop_after <= iter_counter:
                break
        else:
            break